In [1]:
import os
%pwd

'C:\\Users\\manud\\Machine_learning_projects\\Chest Cancer Detection using DL and MLflow\\chest_cancer_classifier\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'C:\\Users\\manud\\Machine_learning_projects\\Chest Cancer Detection using DL and MLflow\\chest_cancer_classifier'

In [5]:
#first of all collect mlflow uri
#this will end up pointing to a tracking server at https://dagshub.com/<username>/<repo>.mlflow automatically == https://dagshub.com/manudevverma429/chest_cancer_classifier.mlflow

import dagshub
dagshub.init(repo_owner='manudevverma429', repo_name='chest_cancer_classifier', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=bde136b8-aad6-4b83-aedb-8bd78400cf0b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=91fc1bb3d7a6b922d28bcc05f78c6215c3f07e5c2343dfe69aebf597853625c3




Accessing as manudevverma429

Initialized MLflow to track repo "manudevverma429/chest_cancer_classifier"

Repository manudevverma429/chest_cancer_classifier initialized!

C:\Users\manud\anaconda3\envs\cancer\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Dataset"),
            mlflow_uri="https://dagshub.com/manudevverma429/chest_cancer_classifier.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [26]:
#initialize my pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-10-12 00:25:22,252: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-12 00:25:22,255: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-12 00:25:22,257: INFO: common: created directory at: artifacts]
ERROR! Session/line number was not unique in database. History logging moved to new session 53



KeyboardInterrupt

